# 🔬 HBV 그룹 비교 분석 - Simulation Data

**Author**: Youngmin (Pranja)

**목적**: 시뮬레이션 데이터로 HBV 상태별 면역세포 분포 비교

---

## 📊 HBV Status (시뮬레이션)

| Status | 설명 |
|--------|------|
| Healthy | 건강인 |
| Chronic_HBV_Tolerant | 만성 HBV (면역 관용기) |
| Chronic_HBV_Active | 만성 HBV (면역 활성기) |
| Resolved_HBV | HBV 완치 |

⚠️ **사전 조건**: `Step1_HBV_simulation.ipynb` 실행 완료

In [ ]:
# @title STEP 1: 환경 설정 및 데이터 로드
import os, sys
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/tahoe-x1'
ANALYSIS_PATH = f'{PROJECT_PATH}/viral_hepatitis/analysis_results'
OUTPUT_PATH = f'{PROJECT_PATH}/viral_hepatitis/comparison_results_simulation'
os.makedirs(OUTPUT_PATH, exist_ok=True)

!pip install scanpy anndata scipy statsmodels seaborn -q

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests
import itertools

# 시뮬레이션 결과 로드 (hbv_tahoe_analysis_*.h5ad)
print('📊 시뮬레이션 분석 결과 검색 중...')
h5ad_files = [f for f in os.listdir(ANALYSIS_PATH) 
              if f.startswith('hbv_tahoe_analysis') and not 'real' in f]

if not h5ad_files:
    raise FileNotFoundError('⚠️ 시뮬레이션 분석 결과가 없습니다. Step1_HBV_simulation.ipynb를 먼저 실행하세요.')

print(f'   발견된 파일: {h5ad_files}')
latest_file = sorted(h5ad_files)[-1]
print(f'   로드할 파일: {latest_file}')

adata = sc.read_h5ad(f'{ANALYSIS_PATH}/{latest_file}')

# 컬럼 확인
status_col = 'hbv_status'

print(f'\n✅ STEP 1 완료!')
print(f'   세포 수: {adata.n_obs:,}')
print(f'   컬럼: {list(adata.obs.columns)}')
print(f'\n📊 HBV Status 분포:')
print(adata.obs[status_col].value_counts())

In [ ]:
# @title STEP 2: 그룹별 세포 분포 비교 (UMAP)
print('📊 그룹별 UMAP 시각화')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sc.pl.umap(adata, color='cell_type', ax=axes[0], show=False,
           title='Cell Types', legend_loc='on data', legend_fontsize=8)

sc.pl.umap(adata, color=status_col, ax=axes[1], show=False,
           title='HBV Status')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/umap_overview.png', dpi=150, bbox_inches='tight')
plt.show()

# Status별 개별 UMAP
statuses = sorted(adata.obs[status_col].unique())
n_status = len(statuses)

fig, axes = plt.subplots(1, n_status, figsize=(5*n_status, 5))
if n_status == 1:
    axes = [axes]

for i, status in enumerate(statuses):
    adata_subset = adata[adata.obs[status_col] == status]
    sc.pl.umap(adata_subset, color='cell_type', ax=axes[i], show=False,
               title=f'{status}\n(n={len(adata_subset):,})', legend_loc='none')

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/umap_by_status.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n✅ STEP 2 완료!')

In [ ]:
# @title STEP 3: 세포 유형별 비율 분석
print('📊 세포 유형별 비율 분석')

cell_type_counts = adata.obs.groupby([status_col, 'cell_type']).size().unstack(fill_value=0)
cell_type_ratios = cell_type_counts.div(cell_type_counts.sum(axis=1), axis=0) * 100

print('\n📋 그룹별 세포 유형 비율 (%):')
print(cell_type_ratios.round(1))

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

cell_type_ratios.plot(kind='bar', stacked=True, ax=axes[0], colormap='tab10')
axes[0].set_title('Cell Type Distribution by HBV Status', fontsize=12)
axes[0].set_xlabel('HBV Status')
axes[0].set_ylabel('Percentage (%)')
axes[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=8)
axes[0].tick_params(axis='x', rotation=45)

sns.heatmap(cell_type_ratios.T, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1],
            cbar_kws={'label': 'Percentage (%)'})
axes[1].set_title('Cell Type Percentage Heatmap', fontsize=12)

plt.tight_layout()
plt.savefig(f'{OUTPUT_PATH}/cell_type_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

cell_type_ratios.to_csv(f'{OUTPUT_PATH}/cell_type_ratios.csv')
print(f'\n✅ STEP 3 완료!')

In [ ]:
# @title STEP 4: 상세 통계 분석
print('📊 상세 통계 분석')

statuses = sorted(adata.obs[status_col].unique())
cell_types = sorted(adata.obs['cell_type'].unique())

# 4-1. 수치 테이블
print('\n' + '='*60)
print('📋 4-1. 세포 수/비율 테이블')
print('='*60)

ratio_table = pd.DataFrame(index=cell_types, columns=statuses)
count_table = pd.DataFrame(index=cell_types, columns=statuses)

for status in statuses:
    status_cells = adata[adata.obs[status_col] == status]
    total = len(status_cells)
    for ct in cell_types:
        count = (status_cells.obs['cell_type'] == ct).sum()
        count_table.loc[ct, status] = count
        ratio_table.loc[ct, status] = round(count / total * 100, 2)

print('\n[세포 수]')
print(count_table.to_string())
print('\n[비율 (%)]')
print(ratio_table.to_string())

count_table.to_csv(f'{OUTPUT_PATH}/detailed_cell_counts.csv')
ratio_table.to_csv(f'{OUTPUT_PATH}/detailed_cell_ratios.csv')

# 4-2. Chi-square 전체
print('\n' + '='*60)
print('📋 4-2. Chi-square 검정 (전체)')
print('='*60)

contingency = adata.obs.groupby([status_col, 'cell_type']).size().unstack(fill_value=0)
chi2_overall, p_overall, dof, _ = stats.chi2_contingency(contingency)
print(f'   Chi-square: {chi2_overall:.2f}')
print(f'   p-value: {p_overall:.2e}')
print(f'   결과: {"✅ 유의함" if p_overall < 0.05 else "유의하지 않음"}')

# 4-3. Pairwise 비교
print('\n' + '='*60)
print('📋 4-3. Pairwise Chi-square (Status 쌍별)')
print('='*60)

pairwise_results = []
for s1, s2 in itertools.combinations(statuses, 2):
    subset = adata[adata.obs[status_col].isin([s1, s2])]
    cont = subset.obs.groupby([status_col, 'cell_type']).size().unstack(fill_value=0)
    if len(cont) == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        pairwise_results.append({
            'Comparison': f'{s1} vs {s2}',
            'Chi2': round(chi2, 2),
            'p-value': p_val,
            'Sig': '✅' if p_val < 0.05 else ''
        })

pairwise_df = pd.DataFrame(pairwise_results)
if len(pairwise_df) > 0:
    _, p_adj, _, _ = multipletests(pairwise_df['p-value'], method='fdr_bh')
    pairwise_df['p-adjusted'] = p_adj
    pairwise_df['FDR Sig'] = ['✅' if p < 0.05 else '' for p in p_adj]

print(pairwise_df.to_string(index=False))
pairwise_df.to_csv(f'{OUTPUT_PATH}/pairwise_chi2.csv', index=False)

# 4-4. 세포 유형별 검정
print('\n' + '='*60)
print('📋 4-4. 세포 유형별 Status 간 검정')
print('='*60)

celltype_stats = []
for ct in cell_types:
    row = {'Cell Type': ct}
    props = []
    for status in statuses:
        s_cells = adata[adata.obs[status_col] == status]
        prop = (s_cells.obs['cell_type'] == ct).mean()
        row[f'{status} (%)'] = round(prop * 100, 2)
        props.append(prop)
    row['Max-Min'] = round((max(props) - min(props)) * 100, 2)
    
    ct_binary = (adata.obs['cell_type'] == ct).astype(int)
    cont = pd.crosstab(adata.obs[status_col], ct_binary)
    if cont.shape[1] == 2:
        chi2, p_val, _, _ = stats.chi2_contingency(cont)
        row['p-value'] = f'{p_val:.2e}'
        row['Sig'] = '✅' if p_val < 0.05 else ''
    celltype_stats.append(row)

celltype_df = pd.DataFrame(celltype_stats)
print(celltype_df.to_string(index=False))
celltype_df.to_csv(f'{OUTPUT_PATH}/celltype_statistics.csv', index=False)

print(f'\n✅ STEP 4 완료!')

In [ ]:
# @title STEP 5: 결과 저장
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

n_sig_pairwise = (pairwise_df['Sig'] == '✅').sum() if len(pairwise_df) > 0 else 0
n_sig_celltype = (celltype_df['Sig'] == '✅').sum()

summary = f"""
================================================================================
🦠 HBV 시뮬레이션 데이터 분석 결과
================================================================================
Author: Youngmin (Pranja)
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

📊 데이터 요약:
   - 세포 수: {adata.n_obs:,}
   - Status 수: {len(statuses)}
   - 세포 유형 수: {len(cell_types)}

📋 HBV Status 분포:
{adata.obs[status_col].value_counts().to_string()}

🔬 통계 결과:
   - 전체 Chi-square: {chi2_overall:.2f}, p = {p_overall:.2e}
   - Pairwise 유의한 쌍: {n_sig_pairwise} / {len(pairwise_df)}
   - 유의한 세포 유형: {n_sig_celltype} / {len(cell_types)}

================================================================================
"""

print(summary)

with open(f'{OUTPUT_PATH}/summary_{timestamp}.txt', 'w', encoding='utf-8') as f:
    f.write(summary)

print('\n🎉 시뮬레이션 데이터 분석 완료!')
print(f'📁 결과 폴더: {OUTPUT_PATH}')